* July 2020
* This notebook was to load the volume database with tweets for review. 
* I already had the tweets from an earlier project https://github.com/jaycee14/tv_sentiment

In [1]:
import psycopg2
from sqlalchemy import create_engine, MetaData,Table, Column, Integer, String, DateTime
from sqlalchemy.orm import mapper,sessionmaker
from sqlalchemy import func
from pathlib import Path
from sqlalchemy.ext.automap import automap_base
from datetime import datetime

In [2]:
from nlp_cmp_utils import get_config, create_db_url

In [3]:
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base

In [4]:
projects_path = Path('/home/jjc/Projects/')
host_addr = "localhost:5432"

### Phase 1 get commetns from tv_sentiment

In [5]:
tv_config = get_config(projects_path / 'tv_sentiment' / 'database.env')

In [6]:
TV_URL = create_db_url(tv_config,host_addr)

In [7]:
engine = create_engine(TV_URL)
meta = MetaData()

In [9]:
conn = engine.raw_connection()
cur = conn.cursor()

In [11]:
sql = """SELECT queries.query_id,queries.show_id, comments.comment_date_utc, query_date, sentiment,
sentiment_score, name, service, comment_text
FROM queries 
JOIN comments ON queries.query_id=comments.query_id 
JOIN shows on queries.show_id = shows.show_id
where sentiment_score > 0.75
"""

In [14]:
res_table =pd.read_sql_query(sql,conn)

In [15]:
pos_selection = res_table.loc[res_table.sentiment=='pos'].sample(1000)
neg_selection = res_table.loc[res_table.sentiment=='neg'].sample(1000)

In [28]:
selection = pd.concat((pos_selection,neg_selection))

In [16]:
conn.close()

### load comments into nlp database
* requires a change of docker container mounted

In [5]:
cmp_config = get_config(projects_path / 'nlp_cmp_human' /'database.env')

In [6]:
CMP_URL = create_db_url(cmp_config,host_addr)

In [7]:
Base= declarative_base()

In [8]:
class Features(Base):
    __tablename__ = 'features'

    id = Column(Integer, primary_key=True)
    feature_text = Column(String)
    type = Column(String)


class Labels(Base):
    __tablename__ = 'labels'

    id = Column(Integer, primary_key=True)
    feature_id = Column(Integer)
    label_text = Column(String)
    entry_id = Column(Integer)


class Entries(Base):
    __tablename__ = 'entries'

    id = Column(Integer, primary_key=True)
    entry_date = Column(DateTime, default=datetime.utcnow)

In [9]:
engine = create_engine(CMP_URL)

In [10]:
Base.metadata.create_all(engine)

In [11]:
Session = sessionmaker(bind=engine)
session = Session()

### select phrases

In [29]:
selection.rename(columns={'comment_text':'feature_text','sentiment':'type'},inplace=True)
selection.head(3)

,query_id,show_id,comment_date_utc,query_date,type,sentiment_score,name,service,feature_text
928,171,7,2020-02-22 19:50:35,2020-02-23 17:01:38.028815,pos,0.895979,picard,amazon,RT @Jwhitbrook: With the new Picard up on Amaz...
5861,670,14,2020-05-01 20:38:06,2020-05-01 23:33:40.790454,pos,0.762469,tiger king,netflix,RT @klaushismydaddy: netflix: i love ALL of my...
3849,458,7,2020-04-16 17:46:05,2020-04-18 11:27:29.562690,pos,0.986387,picard,amazon,Just finished Picard on Amazon Prime. Was not ...


In [30]:
inserts = selection[['feature_text','type']].to_dict(orient='records')

In [34]:
features_to_load=[]
for insert in inserts:
    features_to_load.append(Features(**insert))

In [36]:
features_to_load[0].feature_text

'RT @Jwhitbrook: With the new Picard up on Amazon I can finally get a clean screenshot and ask...\n\nWas anyone else momentarily stunned by ju…'

In [37]:
session.add_all(features_to_load)

In [38]:
session.commit()

In [39]:
for instance in session.query(Features).order_by(Features.id).limit(10):
    print(instance.id, instance.feature_text, instance.type)

1 RT @Jwhitbrook: With the new Picard up on Amazon I can finally get a clean screenshot and ask...

Was anyone else momentarily stunned by ju… pos
2 RT @klaushismydaddy: netflix: i love ALL of my programs equally!!! stranger things, lucifer, tiger king, and *looks at smudged writing on h… pos
3 Just finished Picard on Amazon Prime. Was not expecting to cry. What an excellent show. pos
4 @RachaelCraw @FleurFerris @gabrielletozer Amazon Prime. Where Picard, The Boys and Good Omens can also be found! pos
5 If you haven’t seen the amazon prime show “Upload”, I suggest you go do so. I really enjoyed this movie and it surrounds itself in virtual reality. I have laughed, felt on edge and the story has some twist and turns. Please if you haven’t seen it? Do so! I hope season 2 happens. https://t.co/Mf5bSi pos
6 @brexitblog_info @boblister_poole https://t.co/l2HIKNQ95V

Works both ways pos
7 All 8 episodes of Alex Garland’s DEVS are now available on BBC iPlayer. https://t.co/TzoVZFMD0r pos
8 Vo